# Deep Neural Network for MNIST Classification
### Import relevant packages

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Data

In [6]:
mnist_dataset, mnist_info = tfds.load('mnist', with_info = True, as_supervised = True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000 
#When we're dealing with enormous datasets, we can't shuffle all data at once
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))

W0809 12:06:04.708492  2968 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


## Model
#### Outline Model

In [16]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)), #Input Layer
                            #tf.keras.layers.Flatten(original shape) transforms a tensor into a vector
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #First hidden layer with activation
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #Second hidden layer with activation
                            tf.keras.layers.Dense(output_size, activation='softmax'), #Output layer with softmax
                            ])
#Siehst du? was für ein Kinderspiel, Model eines künstlichen neuronalen Netz zu entwickeln?

#### Choose the optimizer and loss function

In [17]:
#model.compile(optimizer, loss) configures the model of training
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
# binary_crossentropy - refers to the case where we're got binary encoding
# sparse_categorical_crossentropy - applies one-hot encoding.

## Training

In [18]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose=2)
# It will happen inside an epoch
# 1. At the beginning of each epoch, the training loss will be set to 0
# 2. The Algorithm will iterate over a preset number of batches, all from train_data
# 3. The weights and biases will be updated as many times as there are batches
# 4. We will get a value for a loss function, indicating how the training is going
# 5. We will also see a training accuracy
# 6. At the end of the epoch, the algorithm will forward propagete the whole validation set

Epoch 1/5
540/540 - 6s - loss: 0.3262 - accuracy: 0.9080 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 6s - loss: 0.1319 - accuracy: 0.9608 - val_loss: 0.1256 - val_accuracy: 0.9667
Epoch 3/5
540/540 - 6s - loss: 0.0920 - accuracy: 0.9717 - val_loss: 0.0960 - val_accuracy: 0.9743
Epoch 4/5
540/540 - 6s - loss: 0.0710 - accuracy: 0.9785 - val_loss: 0.0758 - val_accuracy: 0.9783
Epoch 5/5
540/540 - 7s - loss: 0.0562 - accuracy: 0.9826 - val_loss: 0.0767 - val_accuracy: 0.9770


## Test the model

In [19]:
#model.evaluate() - returns the loss value and metrics values for the model in 'test mode'
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 2s 2s/step - loss: 0.0899 - accuracy: 0.97 - 2s 2s/step - loss: 0.0899 - accuracy: 0.9721

In [20]:
print("Test Loss: {0: .2f}. Test Accuracy: {1: .2f}%".format(test_loss, test_accuracy*100))

Test Loss:  0.09. Test Accuracy:  97.21%
